In [1]:
import pickle
from langchain_community.retrievers import BM25Retriever # Import is still needed for loading

# Define the file path where the retriever is saved
file_path = "bm25_retriever.pkl"

# Load the BM25Retriever using pickle
with open(file_path, "rb") as f:
    loaded_bm25_retriever = pickle.load(f)

print(f"BM25Retriever loaded from {file_path}")

# You can now use the loaded_bm25_retriever
# query = "lazy dog"
query = "Explain PopularityAdjusted Block Model (PABM)"
relevant_docs = loaded_bm25_retriever.invoke(query)
for doc in relevant_docs:
    print(doc.page_content)

ModuleNotFoundError: No module named 'langchain_core.pydantic_v1'

In [ ]:
## Set up environment variables for LM Studio
import os
os.environ["OPENAI_API_BASE"] = "http://localhost:1234/v1/"
os.environ["OPENAI_API_KEY"] = "test"

In [ ]:
# Set up a LangChain pipeline with prompt template and LLM
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# choose model name exactly as LM Studio exposes it (check LM Studio UI)
# llm = ChatOpenAI(model="qwen/qwen3-4b-thinking-2507", temperature=0.2)  

llm = ChatOpenAI(model="qwen/qwen3-4b-2507", temperature=0)  


template = "Answer the Question based on the context below.\n\nContext: {context}\n\nQ: {question}\nA:"
prompt = PromptTemplate.from_template(template)

# prompt = PromptTemplate(input_variables=["q"], template="Context: Q: {q}\nA:")
chain = prompt | llm | StrOutputParser()

# print(chain.run("Explain recursion in 2 sentences."))

# print(chain.invoke({"context": relevant_docs, "question": "Explain PopularityAdjusted Block Model (PABM)"}))

In [ ]:
##------------------------------------------------------------------------------##

# pip install -qU "langchain[anthropic]" to call the model

from langchain.agents import create_agent

# def get_weather(city: str) -> str:
#     """Get weather for a given city."""
#     return f"It's always sunny in {city}!"

def retrieve_context(query: str) -> str:
    """Retrieve context for a given query using the loaded BM25Retriever."""
    docs = loaded_bm25_retriever.invoke(query)
    return "\n".join(doc.page_content for doc in docs)

agent = create_agent(
    model=llm,
    tools=[retrieve_context],
    system_prompt="You are a helpful assistant. you have access to a tool that retrieves relevant context based on user queries.",
)

# Run the agent
sl = agent.invoke(
    {"messages": [{"role": "user", "content": "who are you?"}]}
)

print("Agent Response:{}".format(sl))
import json
print(type(sl))


# pretty_json_string = json.dumps(sl, indent=4)
# print(pretty_json_string)

##------------------------------------------------------------------------------##

In [ ]:
sl

In [ ]:
sl.get('messages')[1]

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver 

# def get_weather(city: str) -> str:
#     """Get weather for a given city."""
#     return f"It's always sunny in {city}!"

def retrieve_context(query: str) -> str:
    """Retrieve context for a given query using the loaded BM25Retriever."""

    file_path = "bm25_retriever.pkl"

    # Load the BM25Retriever using pickle
    with open(file_path, "rb") as f:
        loaded_bm25_retriever = pickle.load(f)
    docs = loaded_bm25_retriever.invoke(query)
    return "\n".join(doc.page_content for doc in docs)

agent = create_agent(
    model=llm,
    tools=[retrieve_context],
    system_prompt="You are a helpful assistant. you have access to a tool that retrieves relevant context based on user queries.",
    checkpointer=InMemorySaver(),
)

# Run the agent
sl = agent.invoke(
    {"messages": [{"role": "user", "content": "Summarize Our Conversation"}]},
    {"configurable": {"thread_id": "1"}},
)

from langchain_core.runnables import RunnableConfig
config: RunnableConfig = {"configurable": {"thread_id": "1"}}

final_response = agent.invoke({"messages": "hi, my name is Debanjan"}, config)
final_response["messages"][-1].pretty_print()

final_response = agent.invoke({"messages": "Explain PopularityAdjusted Block Model (PABM)"}, config)
final_response["messages"][-1].pretty_print()

final_response = agent.invoke({"messages": "Explain Model Structure and Assumptions"}, config)
final_response["messages"][-1].pretty_print()

final_response = agent.invoke({"messages": "what's my name?"}, config)
final_response["messages"][-1].pretty_print()

# print("Agent Response:{}".format(sl))

# response = sl.get('messages')[-1]

# print("------------------------------")
# print("Agent Response:{}".format(response.content))

In [ ]:
final_response

In [ ]:
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_model
from langgraph.runtime import Runtime
from langchain_core.runnables import RunnableConfig
from typing import Any


@before_model
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]

    if len(messages) <= 3:
        return None  # No changes needed

    first_msg = messages[0]
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    new_messages = [first_msg] + recent_messages

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

agent = create_agent(
    model=llm,
    tools=[retrieve_context],
    middleware=[trim_messages],
    checkpointer=InMemorySaver(),
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}

agent.invoke({"messages": "hi, my name is bob"}, config)
agent.invoke({"messages": "write a short poem about cats"}, config)
agent.invoke({"messages": "now do the same but for dogs"}, config)
final_response = agent.invoke({"messages": "what's my name?"}, config)

final_response["messages"][-1].pretty_print()
"""
================================== Ai Message ==================================

Your name is Bob. You told me that earlier.
If you'd like me to call you a nickname or use a different name, just say the word.
"""